## Resource-Constrained Project Scheduling Problem with Alternative Subgraphs

This notebook demonstrates how to model and solve the Resource-Constrained Project Scheduling Problem with Alternative Process Plan using Constraint Programming with IBM’s CP Optimizer via the [`docplex.cp`](https://ibmdecisionoptimization.github.io/docplex-doc/cp/refman.html) Python API. CP formulation of the problem is inspired by [Jakub Charvat bachelor thesis](https://dspace.cvut.cz/handle/10467/123532).

### Problem Definition

The Resource-Constrained Project Scheduling Problem with Alternative Subgraphs (RCPSP-AS) models a project as a directed acyclic graph $G=(N,A)$, where $N=\{1, \dots, n\}$ is the set of non-preemptive, topologically ordered activities $i, j$, and $A \subset N \times N$ defines the end-start precedence constraints $(i,j)$. Each activity $i$ has a predetermined duration $d_i$ and demands $r_{i,v}$ from a set of renewable resources $R$, where each resource $v \in R$ has a limited capacity $a_v$. The problem's core feature is a set of alternative subgraphs $L$, where each $l \in L$ (or $l' \in L$) is defined by a principal activity $p_l$ and a terminal activity $t_l$. Each subgraph $l$ contains multiple alternative branches $K_{p_l}$, and a branch $k \in K_{p_l}$ (or $k' \in K_{p_l}$) is a subset of activities $N_{b_k} \subset N^a$ starting from a unique branching activity $b_k$, such that $b_k \in N_{b_k} \wedge p_l \notin N_{b_k} \wedge t_l \notin N_{b_k}$ holds. This structure partitions $N$ into alternative activities $N^a = \bigcup N_{b_k}$ and fixed activities $N^f = N \setminus N^a$. Extensions include nested subgraphs and linked branches, the latter defined by a parameter $\kappa_{i,j}=1$ if an activity $i \in N_{b_k}$ is linked to $j \in N_{b_{k'}}$ (where $k \neq k'$).

### CPLEX Formulation

$$
\begin{aligned}
\min \quad
& \mathrm{endOf}(\mathrm{x}_{n-1})
\qquad &\qquad & \text{(1)} \\[2mm]
\text{s.t.} \quad
& \mathrm{presenceOf}(\mathrm{x}_{0}) = 1,
\qquad &\qquad & \text{(2)} \\[1mm]
& \mathrm{ifThen}\!\left(\mathrm{presenceOf}(\mathrm{x}_i) \wedge \mathrm{presenceOf}(\mathrm{x}_j),\; \mathrm{endOf}(\mathrm{x}_i) \le \mathrm{startOf}(\mathrm{x}_j)\right),
\qquad & \forall (i,j)\in A
\quad & \text{(3)} \\[1mm]
& \sum_{k \in K_{p_l}} \mathrm{presenceOf}(\mathrm{x}_{k}) = \mathrm{presenceOf}(\mathrm{x}_{p_l}),
\qquad & \forall l \in L
\quad & \text{(4)} \\[1mm]
& \mathrm{presenceOf}(\mathrm{x}_i) = \left( \sum_{b_k \in M(B_i^{id})} \mathrm{presenceOf}(\mathrm{x}_{b_k}) > 0 \right),
\qquad & \forall i \in N^a
\quad & \text{(5)} \\[1mm]
& \sum_{i \in N} \mathrm{pulse}(\mathrm{x}_i, r_{i,v}) \le a_v,
\qquad & \forall v \in R
\quad & \text{(6)} \\[1mm]
& \text{interval } \mathrm{x}_i\ \text{optional, size}=d_i,
\qquad & \forall i \in N
\quad & \text{(7)}
\end{aligned}
$$

**Objective:**

- **(1)** Minimize the makespan $C_{\max}$, defined as the end time of the sink activity $n-1$.

**Modeling Constraints:**

- **(2)** Ensures the source activity (activity 0) is always selected ("present").
- **(3)** Enforces all precedence relations $(i,j) \in A$. The condition ($\mathrm{endOf}(\mathrm{x}_i) \le \mathrm{startOf}(\mathrm{x}_j)$) is enforced only if both activities $i$ and $j$ are present.
- **(4)** Enforces subgraph branch selection. For each alternative subgraph $l$, the sum of the presence of its branching activities ($k \in K_{p_l}$) must equal the presence of its principal activity $p_l$.
- **(5)** Enforces alternative activity selection. An alternative activity $i \in N^a$ is present if and only if at least one of the branches $k$ it belongs to is selected. Fixed activities ($i \in N^f$) are not constrained by this.
- **(6)** Enforces renewable resource limits. The pulse function models the resource demand $r_{i,v}$ during the execution of a present interval. The sum of all pulses on a resource $v$ must never exceed its capacity $a_v$.

**Variables:**

- **(7)** $\mathrm{x}_i$: An optional interval variable for each activity $i \in N$ with a fixed duration $d_i$.

#### Symbols and Notation

| Symbol / Function | Meaning | $\text{docplex.cp}$ Reference |
| :--- | :--- | :--- |
| $N$ | Set of topologically ordered activities |  |
| $n$ | The total number of activities (sink activity index) |  |
| $A$ | Set of end-start precedence constraints |  |
| $L$ | Set of alternative subgraphs |  |
| $R$ | Set of renewable resources |  |
| $d_i$ | Predetermined duration of activity $i$ |  |
| $r_{i,v}$ | Demand of activity $i$ for renewable resource $v$ |  |
| $a_v$ | Capacity of renewable resource $v$ |  |
| $p_l$ | Principal activity of alternative subgraph $l$ |  |
| $b_k$ | Branching activity of alternative branch $k$ |  |
| $K_{p_l}$ | Set of alternative branches in subgraph $l$ |  |
| $B_i$ | Set of branches activity $i$ belongs to |  |
| $\mathrm{x}_i$ | Optional interval variable for activity $i$ | [docplex.cp.modeler.interval\_var](https://ibmdecisionoptimization.github.io/docplex-doc/cp/docplex.cp.modeler.py.html#docplex.cp.modeler.interval_var) |
| $\mathrm{presenceOf}(\mathrm{x}_i)$ | $1$ if the optional interval $\text{x}_{i}$ (activity $i$) is present, else $0$ | [docplex.cp.modeler.presence\_of](https://ibmdecisionoptimization.github.io/docplex-doc/cp/docplex.cp.modeler.py.html#docplex.cp.modeler.presence_of) |
| $\mathrm{endOf}(\mathrm{x}_n)$ | End time of interval $\text{x}_{n}$ | [docplex.cp.modeler.end\_of](https://ibmdecisionoptimization.github.io/docplex-doc/cp/docplex.cp.modeler.py.html#docplex.cp.modeler.end_of) |
| $\min \mathrm{endOf}(\dots)$ | Objective: minimize makespan | [docplex.cp.modeler.minimize](https://ibmdecisionoptimization.github.io/docplex-doc/cp/docplex.cp.modeler.py.html#docplex.cp.modeler.minimize) |
| $\mathrm{endBeforeStart}(\dots)$ | Non-overlapping precedence constraint | [docplex.cp.modeler.end\_before\_start](https://ibmdecisionoptimization.github.io/docplex-doc/cp/docplex.cp.modeler.py.html#docplex.cp.modeler.end_before_start) |
| $\mathrm{if\_then}(\dots)$ | Logical implication constraint | [docplex.cp.modeler.if\_then](https://ibmdecisionoptimization.github.io/docplex-doc/cp/docplex.cp.modeler.py.html#docplex.cp.modeler.if_then) |
| $\mathrm{pulse}(\mathrm{x}_i, r_{i,v})$ | Time-varying usage (height $r_{i,v}$ when $\text{x}_{i}$ runs) | [docplex.cp.modeler.pulse](https://ibmdecisionoptimization.github.io/docplex-doc/cp/docplex.cp.modeler.py.html#docplex.cp.modeler.pulse) |
| $\sum \mathrm{pulse}(\dots) \le a_v$ | Cumulative resource limit | (Sum of `pulse` expressions) |

### CPLEX Implementation

#### Imports

In [1]:
from docplex.cp.model import CpoModel

# Using ascp package from Charvat's bachelor thesis
from ascp.load_instance import load_instance
from ascp.instance import AslibInstance, Subgraph

#### Reading the data file

In [2]:
instance = load_instance('../../data/rcpspas/ASLIB0/aslib0_0a.RCP')

# create a dictionary to map activity IDs (0-based) to their objects
act_dict = {act.id: act for act in instance.activities}

# build branching activity map links branch_id to branching_activity_id
branch_id_to_activity_id = {
    branch_id: b_k_act_id
    for sub in instance.subgraphs if sub.principal_activity in act_dict
    for b_k_act_id in act_dict[sub.principal_activity].successors if b_k_act_id in act_dict
    for branch_id in act_dict[b_k_act_id].branches.intersection(sub.branches) if branch_id != 0
}

#### Create model and variables

In [3]:
# Create a CP Optimizer model
mdl = CpoModel(name=instance.name)

# (4) create optional interval variable for each activity i with a fixed duration d_i
task_vars = {i: mdl.interval_var(
    name=f"T_{i}", optional=True, size=act.duration) for i, act in act_dict.items()}

#### Add constraints and define objective

In [4]:
# (1) Minimize the makespan (end time of the single sink activity)
mdl.add(mdl.minimize(mdl.end_of(task_vars[len(instance.activities) - 1])))

# (3) Enforces all precedence relations
mdl.add(mdl.if_then(mdl.presence_of(task_vars[act.id]) & mdl.presence_of(task_vars[j]),
                    mdl.end_of(task_vars[act.id]) <= mdl.start_of(task_vars[j]))
        for act in instance.activities for j in act.successors if j in task_vars)

# (4) Enforces subgraph branch selection
mdl.add(mdl.sum(mdl.presence_of(task_vars[s])
                for s in act_dict[sub.principal_activity].successors if s in task_vars) ==
        mdl.presence_of(task_vars[sub.principal_activity])
        for sub in instance.subgraphs if sub.principal_activity in act_dict)

# (2) & (5): Enforces activity selection
source_activity_id = 0
for i, act in act_dict.items():
    if i == source_activity_id:
        # (2) Source activity is present
        mdl.add(mdl.presence_of(task_vars[i]) == 1)
    elif not act.branches or act.branches == {0}:
        mdl.add(mdl.presence_of(task_vars[i]) == mdl.presence_of(
            task_vars[source_activity_id]))  # Fixed activity
    else:
        # (5) Alternative activity presence
        b_vars = [mdl.presence_of(task_vars[branch_id_to_activity_id[b_id]])
                  for b_id in act.branches if b_id in branch_id_to_activity_id]
        mdl.add(mdl.presence_of(task_vars[i]) == (
            mdl.sum(b_vars) > 0) if b_vars else 0)

# (6) Enforces renewable resource limits
mdl.add(mdl.sum(mdl.pulse(task_vars[act.id], act.requirements[v])
                for act in instance.activities if act.requirements[v] > 0) <= capacity
        for v, capacity in enumerate(instance.resources) if capacity > 0)

#### Solve the model

In [5]:
print('Solving model...')
res = mdl.solve(FailLimit=100000,TimeLimit=10)
print('Solution: ')
res.print_solution()

Solving model...
 ! --------------------------------------------------- CP Optimizer 22.1.2.0 --
 ! Minimization problem - 122 variables, 348 constraints
 ! Presolve      : 114 extractables eliminated
 ! FailLimit            = 100000
 ! TimeLimit            = 10
 ! Initial process time : 0.01s (0.01s extraction + 0.00s propagation)
 !  . Log search space  : 846.5 (before), 846.5 (after)
 !  . Memory usage      : 914.8 kB (before), 914.8 kB (after)
 ! Using parallel search with 12 workers.
 ! ----------------------------------------------------------------------------
 !          Best Branches  Non-fixed    W       Branch decision
                        0        122                 -
 + New bound is 65
 ! Using iterative diving.
 *           115      143  0.05s        1      (gap is 43.48%)
 *           109      287  0.05s        1      (gap is 40.37%)
 *           107      419  0.05s        1      (gap is 39.25%)
 *           100      478  0.05s        1      (gap is 35.00%)
         

## Aditional Resources

- [ASLIB dataset](https://www.projectmanagement.ugent.be/research/data)
- [Jakub Charvat bachelor thesis](https://dspace.cvut.cz/handle/10467/123532)